In [2]:
!pip install --upgrade scikit-learn imbalanced-learn



Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.utils import resample
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, Flatten, MaxPooling1D
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from imblearn.over_sampling import SMOTE
from collections import Counter


CNN - bootstrapping

In [1]:
file = ['ant', 'camel', 'ivy', 'jedit', 'log4j', 'lucene', 'poi', 'synapse', 'velocity', 'xalan', 'xerces']
names = {
    'ant': ['1.3', '1.4', '1.5', '1.6', '1.7'],
    'camel': ['1.0', '1.2', '1.4', '1.6'],
    'ivy': ['1.0', '1.1', '1.2'],
    'jedit': ['3.2', '4.0', '4.1', '4.2', '4.3'],
    'log4j': ['1.0', '1.1', '1.2'],
    'lucene': ['2.0', '2.2', '2.4'],
    'poi': ['1.5', '2.0', '2.5', '3.0'],
    'synapse': ['1.0', '1.1', '1.2'],
    'velocity': ['1.4', '1.5', '1.6'],
    'xalan': ['2.4', '2.5', '2.6', '2.7'],
    'xerces': ['1.1', '1.2', '1.3', '1.4.4']
}



In [2]:
def CNN(input_shape):
    model = Sequential()
    model.add(Conv1D(32, kernel_size= 3, activation = 'relu', input_shape =  input_shape))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Dropout(0.1))
    model.add(Conv1D(64, kernel_size  =3, activation = 'relu'))
    model.add(MaxPooling1D(pool_size = 2))
    model.add(Flatten())
    model.add(Dense(64, activation= 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation=  'sigmoid'))

    model.compile(optimizer = 'adam',
                  loss = 'binary_crossentropy',
                  metrics = ['accuracy'])
    return model

In [6]:
def binary(y):
    return (y > 0).astype(int)


for j in range(len(file) - 1):
    for i in range(len(names[file[j]])):

        if i == len(names[file[j]]) - 1:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j+1]}/{file[j+1]}-{names[file[j+1]][0]}.csv"
        elif file[j] == 'xerces' and names[file[j]][i] == '1.4.4':
            break
        else:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i+1]}.csv"


        data_train = pd.read_csv(train_path)
        X_train = data_train.drop(['name', 'bug'], axis=1)
        y_train = binary(data_train['bug'])

        data_test = pd.read_csv(test_path)
        X_test = data_test.drop(['name', 'bug'], axis=1)
        y_test = binary(data_test['bug'])

        
        train_df = pd.concat([X_train.reset_index(drop=True),
                              y_train.reset_index(drop=True)], axis=1)

        major = train_df[train_df['bug'] == 0]
        minor = train_df[train_df['bug'] == 1]

        minority_unsampled = resample(minor,
                                      replace=True,
                                      n_samples=len(major),
                                      random_state=42)
        train_bootstrap = pd.concat([major, minority_unsampled])

        X_train_boot = train_bootstrap.drop('bug', axis=1)
        y_train_boot = train_bootstrap['bug']


        selector = SelectKBest(f_classif, k=10)
        X_train_selected = selector.fit_transform(X_train_boot, y_train_boot)
        X_test_selected = selector.transform(X_test)

        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train_selected)
        X_test_scaled = scaler.transform(X_test_selected)

        X_train_cnn = np.expand_dims(X_train_scaled, axis=2)
        X_test_cnn  = np.expand_dims(X_test_scaled, axis=2)

    
        cnn = CNN(input_shape=(X_train_cnn.shape[1], 1))
        cnn.fit(X_train_cnn, y_train_boot, epochs=10, batch_size=32, verbose=0)


        y_train_pred = (cnn.predict(X_train_cnn) > 0.5).astype(int)
        y_test_pred  = (cnn.predict(X_test_cnn) > 0.5).astype(int)

        print(f"Training on {train_path.split('/')[-1]} & testing on {test_path.split('/')[-1]}: "
              f"f1_score = {f1_score(y_test, y_test_pred):.4f}    "
              f"f1_score_train = {f1_score(y_train_boot, y_train_pred):.4f}")


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Training on ant-1.3.csv & testing on ant-1.4.csv: f1_score = 0.3894    f1_score_train = 0.7911


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Training on ant-1.4.csv & testing on ant-1.5.csv: f1_score = 0.2822    f1_score_train = 0.6713


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Training on xalan-2.5.csv & testing on xalan-2.6.csv: f1_score = 0.5569    f1_score_train = 0.4639


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Training on xalan-2.6.csv & testing on xalan-2.7.csv: f1_score = 0.5382    f1_score_train = 0.7106


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Training on xalan-2.7.csv & testing on xerces-1.1.csv: f1_score = 0.5815    f1_score_train = 0.6875


CNN - smote  - boot

In [7]:


def binary(y):
    return (y > 0).astype(int)

for j in range(len(file) - 1):
    for i in range(len(names[file[j]])):

        if i == len(names[file[j]]) - 1:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j+1]}/{file[j+1]}-{names[file[j+1]][0]}.csv"
        elif file[j] == 'xerces' and names[file[j]][i] == '1.4.4':
            break
        else:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i+1]}.csv"

        data_train = pd.read_csv(train_path)
        X_train = data_train.drop(['name', 'bug'], axis=1)
       
        data_test = pd.read_csv(test_path)
        X_test = data_test.drop(['name', 'bug'], axis=1)


        y_train = binary(data_train['bug'])
        y_test = binary(data_test['bug'])


        selector = SelectKBest(f_classif, k=10)
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)

        
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train_selected)
        X_test = scaler.transform(X_test_selected)

        

        minority_class = min(Counter(y_train), key=Counter(y_train).get)
        minority_count = Counter(y_train)[minority_class]

        if minority_count <= 1:
            X_train_smote, y_train_smote = X_train, y_train
        else:
            k = min(5, minority_count - 1)
            smote = SMOTE(k_neighbors=k, random_state=42)
            X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

    

        train_df = pd.concat([pd.DataFrame(X_train_smote),
                              pd.Series(y_train_smote, name="bug")], axis=1)
        major = train_df[train_df['bug'] == 0]
        minor = train_df[train_df['bug'] == 1]

        minority_unsampled = resample(minor,
                                      replace=True,
                                      n_samples=len(major),
                                      random_state=42)
        train_bootstrap = pd.concat([major, minority_unsampled])

        X_train_boot = train_bootstrap.drop('bug', axis=1).reset_index(drop=True)
        y_train_boot = train_bootstrap['bug'].reset_index(drop=True)

        X_train_cnn = np.expand_dims(X_train_boot, axis=2)
        X_test_cnn  = np.expand_dims(X_test, axis=2)

    
        cnn = CNN(input_shape=(X_train_cnn.shape[1], 1))
        cnn.fit(X_train_cnn, y_train_boot, epochs=10, batch_size=32, verbose=0)

    
        y_train_pred = (cnn.predict(X_train_cnn) > 0.5).astype(int)
        y_test_pred  = (cnn.predict(X_test_cnn) > 0.5).astype(int)

        print(f"Training on {train_path.split('/')[-1]} & testing on {test_path.split('/')[-1]}: "
              f"f1_score = {f1_score(y_test, y_test_pred):.4f}    "
              f"f1_score_train = {f1_score(y_train_boot, y_train_pred):.4f}")



C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Training on ant-1.3.csv & testing on ant-1.4.csv: f1_score = 0.3878    f1_score_train = 0.7850


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Training on ant-1.4.csv & testing on ant-1.5.csv: f1_score = 0.2935    f1_score_train = 0.7398


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Training on xalan-2.5.csv & testing on xalan-2.6.csv: f1_score = 0.6012    f1_score_train = 0.5884


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Training on xalan-2.6.csv & testing on xalan-2.7.csv: f1_score = 0.4437    f1_score_train = 0.7178


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Training on xalan-2.7.csv & testing on xerces-1.1.csv: f1_score = 0.5600    f1_score_train = 0.9296


CNN - smote

In [8]:

def binary(y):
    return (y > 0).astype(int)

for j in range(len(file) - 1):
    for i in range(len(names[file[j]])):

        if i == len(names[file[j]]) - 1:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j+1]}/{file[j+1]}-{names[file[j+1]][0]}.csv"
        elif file[j] == 'xerces' and names[file[j]][i] == '1.4.4':
            break
        else:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i+1]}.csv"

        # Load data
        data_train = pd.read_csv(train_path)
        X_train = data_train.drop(['name', 'bug'], axis=1)
        y_train = binary(data_train['bug'])

        data_test = pd.read_csv(test_path)
        X_test = data_test.drop(['name', 'bug'], axis=1)
        y_test = binary(data_test['bug'])

        # Feature selection
        selector = SelectKBest(f_classif, k=10)
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)

        # Scaling
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train_selected)
        X_test = scaler.transform(X_test_selected)

        # Handle imbalance
        minority_class = min(Counter(y_train), key=Counter(y_train).get)
        minority_count = Counter(y_train)[minority_class]

        if minority_count <= 1:
            X_train_smote, y_train_smote = X_train, y_train
        else:
            k = min(5, minority_count - 1)
            smote = SMOTE(k_neighbors=k, random_state=42)
            X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

        # CNN input formatting
        X_train_cnn = np.expand_dims(X_train_smote, axis=2)
        X_test_cnn  = np.expand_dims(X_test, axis=2)

        # Train CNN
        cnn = CNN(input_shape=(X_train_cnn.shape[1], 1))
        cnn.fit(X_train_cnn, y_train_smote, epochs=10, batch_size=32, verbose=0)

        # Predictions
        y_train_pred = (cnn.predict(X_train_cnn) > 0.5).astype(int)
        y_test_pred  = (cnn.predict(X_test_cnn) > 0.5).astype(int)

        # Scores
        print(f"Training on {train_path.split('/')[-1]} & testing on {test_path.split('/')[-1]}: "
              f"f1_score = {f1_score(y_test, y_test_pred):.4f}    "
              f"f1_score_train = {f1_score(y_train_smote, y_train_pred):.4f}")


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Training on ant-1.3.csv & testing on ant-1.4.csv: f1_score = 0.3673    f1_score_train = 0.7736


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Training on ant-1.4.csv & testing on ant-1.5.csv: f1_score = 0.2718    f1_score_train = 0.6981


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Training on xalan-2.5.csv & testing on xalan-2.6.csv: f1_score = 0.5633    f1_score_train = 0.5985


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Training on xalan-2.6.csv & testing on xalan-2.7.csv: f1_score = 0.4754    f1_score_train = 0.6879


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Training on xalan-2.7.csv & testing on xerces-1.1.csv: f1_score = 0.5143    f1_score_train = 0.8831


Voting 2 mạng CNN khác nhau

In [3]:
def CNN2(input_shape):
    model = Sequential()
    model.add(Conv1D(64, kernel_size=5, activation='relu',padding  = 'same', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))
    model.add(Conv1D(128, kernel_size=3, activation='relu', padding = 'same'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model
class CNNensemble:
    def __init__(self, model1,model2, input_shape, epochs = 10, batch_size=32):
        self.model1 = model1
        self.model2 = model2
        self.input_shape = input_shape
        self.epochs = epochs 
        self.batch_size  = batch_size
        self.best_model = None

    def fit(self, X_train, y_train):
        cnn1 =self.model1(self.input_shape)
        cnn1.fit(X_train, y_train, epochs = self.epochs, batch_size = self.batch_size, verbose = 0)

        cnn2 =self.model2(self.input_shape)
        cnn2.fit(X_train, y_train, epochs = self.epochs, batch_size = self.batch_size, verbose = 0) 
        
        pred1  = (cnn1.predict(X_train) > 0.5).astype(int)
        pred2  = (cnn2.predict(X_train) > 0.5).astype(int)

        f11 = f1_score(y_train, pred1)
        f12 = f1_score(y_train, pred2)

        if f11 >= f12:
            self.best_model = cnn1
        else:
            self.best_model = cnn2
    def predict(self, X):
        return (self.best_model.predict(X)>0.5).astype(int)


voting 2 CNN  - smote  - boot

In [10]:


def binary(y):
    return (y > 0).astype(int)

for j in range(len(file) - 1):
    for i in range(len(names[file[j]])):

        if i == len(names[file[j]]) - 1:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j+1]}/{file[j+1]}-{names[file[j+1]][0]}.csv"
        elif file[j] == 'xerces' and names[file[j]][i] == '1.4.4':
            break
        else:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i+1]}.csv"

        data_train = pd.read_csv(train_path)
        X_train = data_train.drop(['name', 'bug'], axis=1)
       
        data_test = pd.read_csv(test_path)
        X_test = data_test.drop(['name', 'bug'], axis=1)


        y_train = binary(data_train['bug'])
        y_test = binary(data_test['bug'])


        selector = SelectKBest(f_classif, k=10)
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)

        
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train_selected)
        X_test = scaler.transform(X_test_selected)

        

        minority_class = min(Counter(y_train), key=Counter(y_train).get)
        minority_count = Counter(y_train)[minority_class]

        if minority_count <= 1:
            X_train_smote, y_train_smote = X_train, y_train
        else:
            k = min(5, minority_count - 1)
            smote = SMOTE(k_neighbors=k, random_state=42)
            X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

    

        train_df = pd.concat([pd.DataFrame(X_train_smote),
                              pd.Series(y_train_smote, name="bug")], axis=1)
        major = train_df[train_df['bug'] == 0]
        minor = train_df[train_df['bug'] == 1]

        minority_unsampled = resample(minor,
                                      replace=True,
                                      n_samples=len(major),
                                      random_state=42)
        train_bootstrap = pd.concat([major, minority_unsampled])

        X_train_boot = train_bootstrap.drop('bug', axis=1).reset_index(drop=True)
        y_train_boot = train_bootstrap['bug'].reset_index(drop=True)

        X_train_cnn = np.expand_dims(X_train_boot, axis=2)
        X_test_cnn  = np.expand_dims(X_test, axis=2)

    
        ensemble  = CNNensemble(CNN, CNN2,
                                input_shape  = (X_train_cnn.shape[1],1),
                                epochs = 12, batch_size= 32)
        
        ensemble.fit(X_train_cnn, y_train_boot)
    
        y_train_pred = (ensemble.predict(X_train_cnn) > 0.5).astype(int)
        y_test_pred  = (ensemble.predict(X_test_cnn) > 0.5).astype(int)

        print(f"Training on {train_path.split('/')[-1]} & testing on {test_path.split('/')[-1]}: "
              f"f1_score = {f1_score(y_test, y_test_pred):.4f}    "
              f"f1_score_train = {f1_score(y_train_boot, y_train_pred):.4f}")



C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Training on ant-1.3.csv & testing on ant-1.4.csv: f1_score = 0.2947    f1_score_train = 0.8093


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Training on ant-1.4.csv & testing on ant-1.5.csv: f1_score = 0.2842    f1_score_train = 0.7319


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Training on xalan-2.5.csv & testing on xalan-2.6.csv: f1_score = 0.5523    f1_score_train = 0.6547


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Training on xalan-2.6.csv & testing on xalan-2.7.csv: f1_score = 0.5071    f1_score_train = 0.7266


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Training on xalan-2.7.csv & testing on xerces-1.1.csv: f1_score = 0.6463    f1_score_train = 0.9729


EnsemCNN  - smote

In [11]:


def binary(y):
    return (y > 0).astype(int)

for j in range(len(file) - 1):
    for i in range(len(names[file[j]])):

        if i == len(names[file[j]]) - 1:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j+1]}/{file[j+1]}-{names[file[j+1]][0]}.csv"
        elif file[j] == 'xerces' and names[file[j]][i] == '1.4.4':
            break
        else:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i+1]}.csv"

        data_train = pd.read_csv(train_path)
        X_train = data_train.drop(['name', 'bug'], axis=1)
       
        data_test = pd.read_csv(test_path)
        X_test = data_test.drop(['name', 'bug'], axis=1)


        y_train = binary(data_train['bug'])
        y_test = binary(data_test['bug'])


        selector = SelectKBest(f_classif, k=10)
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)

        
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train_selected)
        X_test = scaler.transform(X_test_selected)

        

        minority_class = min(Counter(y_train), key=Counter(y_train).get)
        minority_count = Counter(y_train)[minority_class]

        if minority_count <= 1:
            X_train_smote, y_train_smote = X_train, y_train
        else:
            k = min(5, minority_count - 1)
            smote = SMOTE(k_neighbors=k, random_state=42)
            X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

        X_train_cnn = np.expand_dims(X_train_smote, axis=2)
        X_test_cnn  = np.expand_dims(X_test, axis=2)

    
        ensemble  = CNNensemble(CNN, CNN2,
                                input_shape  = (X_train_cnn.shape[1],1),
                                epochs = 12, batch_size= 32)
        
        ensemble.fit(X_train_cnn, y_train_smote)
    
        y_train_pred = (ensemble.predict(X_train_cnn) > 0.5).astype(int)
        y_test_pred  = (ensemble.predict(X_test_cnn) > 0.5).astype(int)

        print(f"Training on {train_path.split('/')[-1]} & testing on {test_path.split('/')[-1]}: "
              f"f1_score = {f1_score(y_test, y_test_pred):.4f}    "
              f"f1_score_train = {f1_score(y_train_smote, y_train_pred):.4f}")



C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Training on ant-1.3.csv & testing on ant-1.4.csv: f1_score = 0.3725    f1_score_train = 0.8214


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Training on ant-1.4.csv & testing on ant-1.5.csv: f1_score = 0.2857    f1_score_train = 0.6908


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Training on xalan-2.5.csv & testing on xalan-2.6.csv: f1_score = 0.6297    f1_score_train = 0.6327


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Training on xalan-2.6.csv & testing on xalan-2.7.csv: f1_score = 0.5838    f1_score_train = 0.7072


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 991us/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Training on xalan-2.7.csv & testing on xerces-1.1.csv: f1_score = 0.5845    f1_score_train = 0.9666


Ensemble 2 CNN - boot

In [18]:


def binary(y):
    return (y > 0).astype(int)

for j in range(len(file) - 1):
    for i in range(len(names[file[j]])):

        if i == len(names[file[j]]) - 1:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j+1]}/{file[j+1]}-{names[file[j+1]][0]}.csv"
        elif file[j] == 'xerces' and names[file[j]][i] == '1.4.4':
            break
        else:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i+1]}.csv"

        data_train = pd.read_csv(train_path)
        X_train = data_train.drop(['name', 'bug'], axis=1)
       
        data_test = pd.read_csv(test_path)
        X_test = data_test.drop(['name', 'bug'], axis=1)


        y_train = binary(data_train['bug'])
        y_test = binary(data_test['bug'])


        selector = SelectKBest(f_classif, k=10)
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)

        
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train_selected)
        X_test = scaler.transform(X_test_selected)

        

        train_df = pd.concat([pd.DataFrame(X_train),
                              pd.Series(y_train, name="bug")], axis=1)
        major = train_df[train_df['bug'] == 0]
        minor = train_df[train_df['bug'] == 1]

        minority_unsampled = resample(minor,
                                      replace=True,
                                      n_samples=len(major),
                                      random_state=42)
        train_bootstrap = pd.concat([major, minority_unsampled])

        X_train_boot = train_bootstrap.drop('bug', axis=1).reset_index(drop=True)
        y_train_boot = train_bootstrap['bug'].reset_index(drop=True)


        X_train_cnn = np.expand_dims(X_train_boot, axis=2)
        X_test_cnn  = np.expand_dims(X_test, axis=2)

    
        ensemble  = CNNensemble(CNN, CNN2,
                                input_shape  = (X_train_cnn.shape[1],1),
                                epochs = 12, batch_size= 32)
        
        ensemble.fit(X_train_cnn, y_train_boot)
    
        y_train_pred = (ensemble.predict(X_train_cnn) > 0.5).astype(int)
        y_test_pred  = (ensemble.predict(X_test_cnn) > 0.5).astype(int)

        print(f"Training on {train_path.split('/')[-1]} & testing on {test_path.split('/')[-1]}: "
              f"f1_score = {f1_score(y_test, y_test_pred):.4f}    "
              f"f1_score_train = {f1_score(y_train_boot, y_train_pred):.4f}")



C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Training on ant-1.3.csv & testing on ant-1.4.csv: f1_score = 0.3486    f1_score_train = 0.7982


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Training on ant-1.4.csv & testing on ant-1.5.csv: f1_score = 0.3077    f1_score_train = 0.7340


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Training on xalan-2.5.csv & testing on xalan-2.6.csv: f1_score = 0.5637    f1_score_train = 0.5990


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Training on xalan-2.6.csv & testing on xalan-2.7.csv: f1_score = 0.5811    f1_score_train = 0.7209


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Training on xalan-2.7.csv & testing on xerces-1.1.csv: f1_score = 0.4156    f1_score_train = 0.7000


Thử nghiệm: giải quyết triệt để vấn đề chênh lệch phân phối giữa tập train và test khi huấn luyện chéo version


In [5]:
def Coral(X_train, X_test, reg   = 1e-6):
    #covariance source va target
    cov_train  = np.cov(X_train, rowvar = False) + reg * np.eye(X_train.shape[1]) 
    cov_test = np.cov(X_test, rowvar= False) + reg*np.eye(X_test.shape[1])
    # Whitening: source -> identity
    U_train, S_train, _ = np.linalg.svd(cov_train)
    cov_train_inv_sqrt = (U_train @ np.diag(1.0 / np.sqrt(S_train)) @ U_train.T)

    # Coloring: identity -> target
    U_test, S_test, _ = np.linalg.svd(cov_test)
    cov_test_sqrt = (U_test @ np.diag(np.sqrt(S_test)) @ U_test.T)

    # Transform source
    X_train_new = (X_train @ cov_train_inv_sqrt) @ cov_test_sqrt
    return X_train_new

Ta thử lại với mô hình 1 CNN


In [ ]:
#smote - coral
def binary(y):
    return (y > 0).astype(int)

for j in range(len(file) - 1):
    for i in range(len(names[file[j]])):

        if i == len(names[file[j]]) - 1:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j+1]}/{file[j+1]}-{names[file[j+1]][0]}.csv"
        elif file[j] == 'xerces' and names[file[j]][i] == '1.4.4':
            break
        else:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i+1]}.csv"

        # Load data
        data_train = pd.read_csv(train_path)
        X_train = data_train.drop(['name', 'bug'], axis=1)
        y_train = binary(data_train['bug'])

        data_test = pd.read_csv(test_path)
        X_test = data_test.drop(['name', 'bug'], axis=1)
        y_test = binary(data_test['bug'])

        # Feature selection
        selector = SelectKBest(f_classif, k=10)
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)

        # Scaling
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train_selected)
        X_test = scaler.transform(X_test_selected)

        if i == len(names[file[j]]) - 1:
            
            X_train = Coral(X_train, X_test)

        # Handle imbalance
        minority_class = min(Counter(y_train), key=Counter(y_train).get)
        minority_count = Counter(y_train)[minority_class]

        if minority_count <= 1:
            X_train_smote, y_train_smote = X_train, y_train
        else:
            k = min(5, minority_count - 1)
            smote = SMOTE(k_neighbors=k, random_state=42)
            X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)


        # CNN input formatting
        X_train_cnn = np.expand_dims(X_train_smote, axis=2)
        X_test_cnn  = np.expand_dims(X_test, axis=2)

        # Train CNN
        cnn = CNN(input_shape=(X_train_cnn.shape[1], 1))
        cnn.fit(X_train_cnn, y_train_smote, epochs=10, batch_size=32, verbose=0)

        # Predictions
        y_train_pred = (cnn.predict(X_train_cnn) > 0.5).astype(int)
        y_test_pred  = (cnn.predict(X_test_cnn) > 0.5).astype(int)

        # Scores
        print(f"Training on {train_path.split('/')[-1]} & testing on {test_path.split('/')[-1]}: "
              f"f1_score = {f1_score(y_test, y_test_pred):.4f}    "
              f"f1_score_train = {f1_score(y_train_smote, y_train_pred):.4f}")


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Training on ant-1.3.csv & testing on ant-1.4.csv: f1_score = 0.3800    f1_score_train = 0.8161


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Training on ant-1.4.csv & testing on ant-1.5.csv: f1_score = 0.2945    f1_score_train = 0.6621


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Training on xalan-2.5.csv & testing on xalan-2.6.csv: f1_score = 0.6020    f1_score_train = 0.5837


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Training on xalan-2.6.csv & testing on xalan-2.7.csv: f1_score = 0.4162    f1_score_train = 0.6692


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Training on xalan-2.7.csv & testing on xerces-1.1.csv: f1_score = 0.4324    f1_score_train = 0.9374


CNN - boot - coral

In [11]:

def binary(y):
    return (y > 0).astype(int)

for j in range(len(file) - 1):
    for i in range(len(names[file[j]])):

        if i == len(names[file[j]]) - 1:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j+1]}/{file[j+1]}-{names[file[j+1]][0]}.csv"
        elif file[j] == 'xerces' and names[file[j]][i] == '1.4.4':
            break
        else:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i+1]}.csv"

        # Load data
        data_train = pd.read_csv(train_path)
        X_train = data_train.drop(['name', 'bug'], axis=1)
        y_train = binary(data_train['bug'])

        data_test = pd.read_csv(test_path)
        X_test = data_test.drop(['name', 'bug'], axis=1)
        y_test = binary(data_test['bug'])

        # Feature selection
        selector = SelectKBest(f_classif, k=10)
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)

        # Scaling
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train_selected)
        X_test = scaler.transform(X_test_selected)

        if i == len(names[file[j]]) - 1:
            
            X_train = Coral(X_train, X_test)

        train_df = pd.concat([pd.DataFrame(X_train),
                              pd.Series(y_train, name="bug")], axis=1)
        major = train_df[train_df['bug'] == 0]
        minor = train_df[train_df['bug'] == 1]

        minority_unsampled = resample(minor,
                                      replace=True,
                                      n_samples=len(major),
                                      random_state=42)
        train_bootstrap = pd.concat([major, minority_unsampled])

        X_train_boot = train_bootstrap.drop('bug', axis=1).reset_index(drop=True)
        y_train_boot = train_bootstrap['bug'].reset_index(drop=True)

       
        X_train_cnn = np.expand_dims(X_train_boot, axis=2)
        X_test_cnn  = np.expand_dims(X_test, axis=2)

        
        cnn = CNN(input_shape=(X_train_cnn.shape[1], 1))
        cnn.fit(X_train_cnn, y_train_boot, epochs=10, batch_size=32, verbose=0)

        
        y_train_pred = (cnn.predict(X_train_cnn) > 0.5).astype(int)
        y_test_pred  = (cnn.predict(X_test_cnn) > 0.5).astype(int)

    
        print(f"Training on {train_path.split('/')[-1]} & testing on {test_path.split('/')[-1]}: "
              f"f1_score = {f1_score(y_test, y_test_pred):.4f}    "
              f"f1_score_train = {f1_score(y_train_boot, y_train_pred):.4f}")


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Training on ant-1.3.csv & testing on ant-1.4.csv: f1_score = 0.3922    f1_score_train = 0.7619


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Training on ant-1.4.csv & testing on ant-1.5.csv: f1_score = 0.2968    f1_score_train = 0.6783


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Training on xalan-2.5.csv & testing on xalan-2.6.csv: f1_score = 0.5601    f1_score_train = 0.5347


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Training on xalan-2.6.csv & testing on xalan-2.7.csv: f1_score = 0.5339    f1_score_train = 0.7258


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Training on xalan-2.7.csv & testing on xerces-1.1.csv: f1_score = 0.4000    f1_score_train = 0.7000


CNN - boot - smote - coral

In [14]:


def binary(y):
    return (y > 0).astype(int)

for j in range(len(file) - 1):
    for i in range(len(names[file[j]])):

        if i == len(names[file[j]]) - 1:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j+1]}/{file[j+1]}-{names[file[j+1]][0]}.csv"
        elif file[j] == 'xerces' and names[file[j]][i] == '1.4.4':
            break
        else:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i+1]}.csv"

        data_train = pd.read_csv(train_path)
        X_train = data_train.drop(['name', 'bug'], axis=1)
       
        data_test = pd.read_csv(test_path)
        X_test = data_test.drop(['name', 'bug'], axis=1)


        y_train = binary(data_train['bug'])
        y_test = binary(data_test['bug'])


        selector = SelectKBest(f_classif, k=10)
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)

        
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train_selected)
        X_test = scaler.transform(X_test_selected)

        
        if i == len(names[file[j]]) - 1:
            X_train  = Coral(X_train, X_test)
        

        minority_class = min(Counter(y_train), key=Counter(y_train).get)
        minority_count = Counter(y_train)[minority_class]

        if minority_count <= 1:
            X_train_smote, y_train_smote = X_train, y_train
        else:
            k = min(5, minority_count - 1)
            smote = SMOTE(k_neighbors=k, random_state=42)
            X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

    

        train_df = pd.concat([pd.DataFrame(X_train_smote),
                              pd.Series(y_train_smote, name="bug")], axis=1)
        major = train_df[train_df['bug'] == 0]
        minor = train_df[train_df['bug'] == 1]

        minority_unsampled = resample(minor,
                                      replace=True,
                                      n_samples=len(major),
                                      random_state=42)
        train_bootstrap = pd.concat([major, minority_unsampled])

        X_train_boot = train_bootstrap.drop('bug', axis=1).reset_index(drop=True)
        y_train_boot = train_bootstrap['bug'].reset_index(drop=True)

        X_train_cnn = np.expand_dims(X_train_boot, axis=2)
        X_test_cnn  = np.expand_dims(X_test, axis=2)

    
        cnn = CNN(input_shape=(X_train_cnn.shape[1], 1))
        cnn.fit(X_train_cnn, y_train_boot, epochs=10, batch_size=32, verbose=0)

    
        y_train_pred = (cnn.predict(X_train_cnn) > 0.5).astype(int)
        y_test_pred  = (cnn.predict(X_test_cnn) > 0.5).astype(int)

        print(f"Training on {train_path.split('/')[-1]} & testing on {test_path.split('/')[-1]}: "
              f"f1_score = {f1_score(y_test, y_test_pred):.4f}    "
              f"f1_score_train = {f1_score(y_train_boot, y_train_pred):.4f}")



C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Training on ant-1.3.csv & testing on ant-1.4.csv: f1_score = 0.3962    f1_score_train = 0.7964


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Training on ant-1.4.csv & testing on ant-1.5.csv: f1_score = 0.2710    f1_score_train = 0.7027


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Training on xalan-2.5.csv & testing on xalan-2.6.csv: f1_score = 0.6020    f1_score_train = 0.6455


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Training on xalan-2.6.csv & testing on xalan-2.7.csv: f1_score = 0.4162    f1_score_train = 0.7007


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Training on xalan-2.7.csv & testing on xerces-1.1.csv: f1_score = 0.4400    f1_score_train = 0.9535


Ensemble 2 CNN - smote - coral


In [ ]:


def binary(y):
    return (y > 0).astype(int)

for j in range(len(file) - 1):
    for i in range(len(names[file[j]])):

        if i == len(names[file[j]]) - 1:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j+1]}/{file[j+1]}-{names[file[j+1]][0]}.csv"
        elif file[j] == 'xerces' and names[file[j]][i] == '1.4.4':
            break
        else:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i+1]}.csv"

        data_train = pd.read_csv(train_path)
        X_train = data_train.drop(['name', 'bug'], axis=1)
       
        data_test = pd.read_csv(test_path)
        X_test = data_test.drop(['name', 'bug'], axis=1)


        y_train = binary(data_train['bug'])
        y_test = binary(data_test['bug'])


        selector = SelectKBest(f_classif, k=10)
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)

        
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train_selected)
        X_test = scaler.transform(X_test_selected)

        if i == len(names[file[j]]) - 1:
            
            X_train = Coral(X_train, X_test)


        

        minority_class = min(Counter(y_train), key=Counter(y_train).get)
        minority_count = Counter(y_train)[minority_class]

        if minority_count <= 1:
            X_train_smote, y_train_smote = X_train, y_train
        else:
            k = min(5, minority_count - 1)
            smote = SMOTE(k_neighbors=k, random_state=42)
            X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

        X_train_cnn = np.expand_dims(X_train_smote, axis=2)
        X_test_cnn  = np.expand_dims(X_test, axis=2)

    
        ensemble  = CNNensemble(CNN, CNN2,
                                input_shape  = (X_train_cnn.shape[1],1),
                                epochs = 12, batch_size= 32)
        
        ensemble.fit(X_train_cnn, y_train_smote)
    
        y_train_pred = (ensemble.predict(X_train_cnn) > 0.5).astype(int)
        y_test_pred  = (ensemble.predict(X_test_cnn) > 0.5).astype(int)

        print(f"Training on {train_path.split('/')[-1]} & testing on {test_path.split('/')[-1]}: "
              f"f1_score = {f1_score(y_test, y_test_pred):.4f}    "
              f"f1_score_train = {f1_score(y_train_smote, y_train_pred):.4f}")



C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Training on ant-1.3.csv & testing on ant-1.4.csv: f1_score = 0.3600    f1_score_train = 0.8546


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Training on ant-1.4.csv & testing on ant-1.5.csv: f1_score = 0.2985    f1_score_train = 0.7029


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Training on xalan-2.5.csv & testing on xalan-2.6.csv: f1_score = 0.6512    f1_score_train = 0.6486


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Training on xalan-2.6.csv & testing on xalan-2.7.csv: f1_score = 0.5339    f1_score_train = 0.6956


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 897us/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Training on xalan-2.7.csv & testing on xerces-1.1.csv: f1_score = 0.4324    f1_score_train = 0.9543


Ensemble 2 CNN  - smote - coral - boot

In [16]:


def binary(y):
    return (y > 0).astype(int)

for j in range(len(file) - 1):
    for i in range(len(names[file[j]])):

        if i == len(names[file[j]]) - 1:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j+1]}/{file[j+1]}-{names[file[j+1]][0]}.csv"
        elif file[j] == 'xerces' and names[file[j]][i] == '1.4.4':
            break
        else:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i+1]}.csv"

        data_train = pd.read_csv(train_path)
        X_train = data_train.drop(['name', 'bug'], axis=1)
       
        data_test = pd.read_csv(test_path)
        X_test = data_test.drop(['name', 'bug'], axis=1)


        y_train = binary(data_train['bug'])
        y_test = binary(data_test['bug'])


        selector = SelectKBest(f_classif, k=10)
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)

        
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train_selected)
        X_test = scaler.transform(X_test_selected)

        X_train = Coral(X_train, X_test)

        

        minority_class = min(Counter(y_train), key=Counter(y_train).get)
        minority_count = Counter(y_train)[minority_class]

        if minority_count <= 1:
            X_train_smote, y_train_smote = X_train, y_train
        else:
            k = min(5, minority_count - 1)
            smote = SMOTE(k_neighbors=k, random_state=42)
            X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

    

        train_df = pd.concat([pd.DataFrame(X_train_smote),
                              pd.Series(y_train_smote, name="bug")], axis=1)
        major = train_df[train_df['bug'] == 0]
        minor = train_df[train_df['bug'] == 1]

        minority_unsampled = resample(minor,
                                      replace=True,
                                      n_samples=len(major),
                                      random_state=42)
        train_bootstrap = pd.concat([major, minority_unsampled])

        X_train_boot = train_bootstrap.drop('bug', axis=1).reset_index(drop=True)
        y_train_boot = train_bootstrap['bug'].reset_index(drop=True)

        X_train_cnn = np.expand_dims(X_train_boot, axis=2)
        X_test_cnn  = np.expand_dims(X_test, axis=2)

    
        ensemble  = CNNensemble(CNN, CNN2,
                                input_shape  = (X_train_cnn.shape[1],1),
                                epochs = 12, batch_size= 32)
        
        ensemble.fit(X_train_cnn, y_train_boot)
    
        y_train_pred = (ensemble.predict(X_train_cnn) > 0.5).astype(int)
        y_test_pred  = (ensemble.predict(X_test_cnn) > 0.5).astype(int)

        print(f"Training on {train_path.split('/')[-1]} & testing on {test_path.split('/')[-1]}: "
              f"f1_score = {f1_score(y_test, y_test_pred):.4f}    "
              f"f1_score_train = {f1_score(y_train_boot, y_train_pred):.4f}")





C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Training on ant-1.3.csv & testing on ant-1.4.csv: f1_score = 0.3333    f1_score_train = 0.8559


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Training on ant-1.4.csv & testing on ant-1.5.csv: f1_score = 0.3140    f1_score_train = 0.7067


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Training on xalan-2.5.csv & testing on xalan-2.6.csv: f1_score = 0.6459    f1_score_train = 0.6970


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Training on xalan-2.6.csv & testing on xalan-2.7.csv: f1_score = 0.4500    f1_score_train = 0.7277


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 941us/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Training on xalan-2.7.csv & testing on xerces-1.1.csv: f1_score = 0.4258    f1_score_train = 0.9632


DNN

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

class DNN:
    def __init__(self, input_dim, hidden_layers,
                 activation="relu", output_activation="sigmoid",
                 learning_rate=0.00001,alpha=0.25, gamma=2.0, loss=None, optimizer="adam"):

        self.input_dim = input_dim
        self.hidden_layers = hidden_layers
        self.activation = activation
        self.output_activation = output_activation
        self.learning_rate = learning_rate
        self.alpha =  alpha
        self.gamma = gamma
        self.optimizer = optimizer
        self.loss = loss if loss is not None else self.focal_loss
        self.model = self._build_model()

    def focal_loss(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)

        cross_entropy = -(
            y_true * tf.math.log(y_pred) +
            (1 - y_true) * tf.math.log(1 - y_pred)
        )

        prob_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        focal_weight = self.alpha * tf.pow(1 - prob_t, self.gamma)

        return tf.reduce_mean(focal_weight * cross_entropy)

    def _get_optimizer(self):
        if isinstance(self.optimizer, str):
            opt = self.optimizer.lower()
            if opt == "adam":
                return keras.optimizers.Adam(learning_rate=self.learning_rate)
            elif opt == "sgd":
                return keras.optimizers.SGD(learning_rate=self.learning_rate, momentum=0.9)
            elif opt == "rmsprop":
                return keras.optimizers.RMSprop(learning_rate=self.learning_rate)
            elif opt == "adagrad":
                return keras.optimizers.Adagrad(learning_rate=self.learning_rate)
            else:
                raise ValueError(f"Optimizer '{self.optimizer}' không được hỗ trợ!")
        else:
            return self.optimizer

    def _build_model(self):
        model = keras.Sequential()
        model.add(layers.Input(shape=(self.input_dim,)))
        for units in self.hidden_layers:
            model.add(layers.Dense(units, activation=self.activation))
        model.add(layers.Dense(1, activation=self.output_activation))

        model.compile(
            optimizer=self._get_optimizer(),
            loss=self.loss,
            metrics=["accuracy"]
        )
        return model

    def fit(self, X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1):
        return self.model.fit(
            X_train, y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=validation_split,
            verbose=verbose,
            shuffle = True
        )

    def evaluate(self, X_test, y_test):
        return self.model.evaluate(X_test, y_test, verbose=0)

    def predict(self, X):
        return self.model.predict(X)


In [4]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
class DNN:
    def __init__(self, input_dim, hidden_layers=[128, 64, 32],
                 activation="relu", output_activation="sigmoid",
                 learning_rate=0.01, loss="binary_crossentropy"):
        self.input_dim = input_dim
        self.hidden_layers = hidden_layers
        self.activation = activation
        self.output_activation = output_activation
        self.learning_rate = learning_rate
        self.loss = loss
        self.model = self._build_model()

    def _build_model(self):
        model = keras.Sequential()
        model.add(layers.Input(shape=(self.input_dim,)))
        for units in self.hidden_layers:
            model.add(layers.Dense(units, activation=self.activation))
        model.add(layers.Dense(1, activation=self.output_activation))
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=self.learning_rate),
            loss=self.loss,
            metrics=["accuracy"]
        )
        return model
    def fit(self, X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1):
        return self.model.fit(X_train, y_train,
                              epochs=epochs,
                              batch_size=batch_size,
                              validation_split=validation_split,
                              verbose=verbose)

    def evaluate(self, X_test, y_test):
        return self.model.evaluate(X_test, y_test, verbose=0)

    def predict(self, X):
        return self.model.predict(X)


In [24]:


def binary(y):
    return (y > 0).astype(int)

for j in range(len(file) - 1):
    for i in range(len(names[file[j]])):
        if i == len(names[file[j]]) - 1:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j+1]}/{file[j+1]}-{names[file[j+1]][0]}.csv"
        elif file[j] == 'xerces' and names[file[j]][i] == '1.4.4':
            break
        else:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i+1]}.csv"

        data_train = pd.read_csv(train_path)
        X_train = data_train.drop(['name', 'bug'], axis=1)
        y_train = binary(data_train['bug'])

        data_test = pd.read_csv(test_path)
        X_test = data_test.drop(['name', 'bug'], axis=1)
        y_test = binary(data_test['bug'])

        # Feature selection
        selector = SelectKBest(f_classif, k=10)
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)

        # Scaling
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train_selected)
        X_test_scaled = scaler.transform(X_test_selected)

        if i == len(names[file[j]]) - 1:
            X_train_scaled =  Coral(X_train_scaled, X_test_scaled)

        # Oversampling
        minority_class = min(Counter(y_train), key=Counter(y_train).get)
        minority_count = Counter(y_train)[minority_class]
        if minority_count <= 1:
            X_train_over, y_train_over = X_train_scaled, y_train
        else:
            k = min(5, minority_count - 1)
            oversample = SMOTE(k_neighbors=k)
            X_train_over, y_train_over = oversample.fit_resample(X_train_scaled, y_train)

        # Train DNN
        dnn = DNN(input_dim=X_train_over.shape[1], hidden_layers=[128, 64, 32])
        dnn.fit(X_train_over, y_train_over, epochs=30, batch_size=32, verbose=0)

        # Predict
        y_test_pred = (dnn.predict(X_test_scaled) > 0.5).astype(int)
        y_train_pred = (dnn.predict(X_train_over) > 0.5).astype(int)

        print(f"Training on {train_path.split('/')[-1]} & testing on {test_path.split('/')[-1]}: "
              f"f1_score = {f1_score(y_test, y_test_pred):.4f}    "
              f"f1_score_train = {f1_score(y_train_over, y_train_pred):.4f}")


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Training on ant-1.3.csv & testing on ant-1.4.csv: f1_score = 0.3077    f1_score_train = 0.9202
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Training on ant-1.4.csv & testing on ant-1.5.csv: f1_score = 0.2581    f1_score_train = 0.8949
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Training on ant-1.5.csv & testing on ant-1.6.csv: f1_score = 0.4626    f1_score_train = 0.8000
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Training on ant-1.6.csv & testing on ant-1.7.csv: f1_score = 0.5723    f1_score_train = 0.7659
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step
Training on ant-1.7.csv & testing on camel-1.0.csv: f1_score = 0.1087    f1_score_train = 0.7761
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 915us/step
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Training on camel-1.0.csv & testing on camel-1.

In [ ]:


def binary(y):
    return (y > 0).astype(int)

for j in range(len(file) - 1):
    for i in range(len(names[file[j]])):
        if i == len(names[file[j]]) - 1:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j+1]}/{file[j+1]}-{names[file[j+1]][0]}.csv"
        elif file[j] == 'xerces' and names[file[j]][i] == '1.4.4':
            break
        else:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i+1]}.csv"

        data_train = pd.read_csv(train_path)
        X_train = data_train.drop(['name', 'bug'], axis=1)
        y_train = binary(data_train['bug'])

        data_test = pd.read_csv(test_path)
        X_test = data_test.drop(['name', 'bug'], axis=1)
        y_test = binary(data_test['bug'])

        # Feature selection
        selector = SelectKBest(f_classif, k=10)
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)

        # Scaling
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train_selected)
        X_test_scaled = scaler.transform(X_test_selected)

        if i == len(names[file[j]]) - 1:
            X_train_scaled =  Coral(X_train_scaled, X_test_scaled)

        train_df = pd.concat([pd.DataFrame(X_train_scaled),
                              pd.Series(y_train, name="bug")], axis=1)
        major = train_df[train_df['bug'] == 0]
        minor = train_df[train_df['bug'] == 1]

        minority_unsampled = resample(minor,
                                      replace=True,
                                      n_samples=len(major),
                                      random_state=42)
        train_bootstrap = pd.concat([major, minority_unsampled])

        X_train_boot = train_bootstrap.drop('bug', axis=1).reset_index(drop=True)
        y_train_boot = train_bootstrap['bug'].reset_index(drop=True)

        # Train DNN
        dnn = DNN(input_dim=X_train_boot.shape[1], hidden_layers=[128, 64, 32])
        dnn.fit(X_train_boot, y_train_boot, epochs=30, batch_size=32, verbose=0)

        # Predict
        y_test_pred = (dnn.predict(X_test_scaled) > 0.5).astype(int)
        y_train_pred = (dnn.predict(X_train_boot) > 0.5).astype(int)

        print(f"Training on {train_path.split('/')[-1]} & testing on {test_path.split('/')[-1]}: "
              f"f1_score = {f1_score(y_test, y_test_pred):.4f}    "
              f"f1_score_train = {f1_score(y_train_boot, y_train_pred):.4f}")


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Training on ant-1.3.csv & testing on ant-1.4.csv: f1_score = 0.2895    f1_score_train = 0.8821
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Training on ant-1.4.csv & testing on ant-1.5.csv: f1_score = 0.3400    f1_score_train = 0.9110
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Training on ant-1.5.csv & testing on ant-1.6.csv: f1_score = 0.4690    f1_score_train = 0.8475
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Training on ant-1.6.csv & testing on ant-1.7.csv: f1_score = 0.5563    f1_score_train = 0.8393
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 686us/step
Training on ant-1.7.csv & testing on camel-1.0.csv: f1_score = 0.1333    f1_score_train = 0.7755
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Training on camel-1.0.csv & testing on camel-1.2

In [26]:


def binary(y):
    return (y > 0).astype(int)

for j in range(len(file) - 1):
    for i in range(len(names[file[j]])):
        if i == len(names[file[j]]) - 1:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j+1]}/{file[j+1]}-{names[file[j+1]][0]}.csv"
        elif file[j] == 'xerces' and names[file[j]][i] == '1.4.4':
            break
        else:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i+1]}.csv"

        data_train = pd.read_csv(train_path)
        X_train = data_train.drop(['name', 'bug'], axis=1)
        y_train = binary(data_train['bug'])

        data_test = pd.read_csv(test_path)
        X_test = data_test.drop(['name', 'bug'], axis=1)
        y_test = binary(data_test['bug'])

        # Feature selection
        selector = SelectKBest(f_classif, k=10)
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)


        # Scaling
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train_selected)
        X_test_scaled = scaler.transform(X_test_selected)

        if i == len(names[file[j]]) - 1:
            X_train_scaled =  Coral(X_train_scaled, X_test_scaled)

        # Oversampling
        minority_class = min(Counter(y_train), key=Counter(y_train).get)
        minority_count = Counter(y_train)[minority_class]
        if minority_count <= 1:
            X_train_over, y_train_over = X_train_scaled, y_train
        else:
            k = min(5, minority_count - 1)
            oversample = SMOTE(k_neighbors=k)
            X_train_over, y_train_over = oversample.fit_resample(X_train_scaled, y_train)

        train_df = pd.concat([pd.DataFrame(X_train_over),
                              pd.Series(y_train, name="bug")], axis=1)
        major = train_df[train_df['bug'] == 0]
        minor = train_df[train_df['bug'] == 1]

        minority_unsampled = resample(minor,
                                      replace=True,
                                      n_samples=len(major),
                                      random_state=42)
        train_bootstrap = pd.concat([major, minority_unsampled])

        X_train_boot = train_bootstrap.drop('bug', axis=1).reset_index(drop=True)
        y_train_boot = train_bootstrap['bug'].reset_index(drop=True)

        # Train DNN
        dnn = DNN(input_dim=X_train_over.shape[1], hidden_layers=[128, 64, 32])
        dnn.fit(X_train_boot, y_train_boot, epochs=30, batch_size=32, verbose=0)

        # Predict
        y_test_pred = (dnn.predict(X_test_scaled) > 0.5).astype(int)
        y_train_pred = (dnn.predict(X_train_boot) > 0.5).astype(int)

        print(f"Training on {train_path.split('/')[-1]} & testing on {test_path.split('/')[-1]}: "
              f"f1_score = {f1_score(y_test, y_test_pred):.4f}    "
              f"f1_score_train = {f1_score(y_train_boot, y_train_pred):.4f}")


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Training on ant-1.3.csv & testing on ant-1.4.csv: f1_score = 0.2090    f1_score_train = 0.8912
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Training on ant-1.4.csv & testing on ant-1.5.csv: f1_score = 0.2388    f1_score_train = 0.9078
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Training on ant-1.5.csv & testing on ant-1.6.csv: f1_score = 0.4029    f1_score_train = 0.8276
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Training on ant-1.6.csv & testing on ant-1.7.csv: f1_score = 0.5434    f1_score_train = 0.8065
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 627us/step
Training on ant-1.7.csv & testing on camel-1.0.csv: f1_score = 0.1333    f1_score_train = 0.8112
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 924us/step
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Training on camel-1.0.csv & testing on camel-1.

Stacking 2 mô hình DNN và lấy output của chúng làm input cho một meta model(đã ghi lại)

In [13]:
from sklearn.linear_model import LogisticRegression

class DNNStacking:
    def __init__(self, model1_params, model2_params):
        self.model1 = DNN(**model1_params)
        self.model2 = DNN(**model2_params)
        self.meta = LogisticRegression()

    def fit(self, X_train, y_train, epochs=50, batch_size=32, verbose=1):
        # train base models
        self.model1.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
        self.model2.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)

        # tạo features mới từ output 2 DNN
        pred1 = self.model1.predict(X_train).reshape(-1, 1)
        pred2 = self.model2.predict(X_train).reshape(-1, 1)
        X_meta = np.hstack([pred1, pred2])

        # train meta model
        self.meta.fit(X_meta, y_train)

    def predict(self, X):
        pred1 = self.model1.predict(X).reshape(-1, 1)
        pred2 = self.model2.predict(X).reshape(-1, 1)
        X_meta = np.hstack([pred1, pred2])
        return self.meta.predict_proba(X_meta)[:, 1]  # xác suất cho class=1


In [14]:
params1 = {
    "input_dim": 10,
    "hidden_layers": [128, 64, 32],
    "learning_rate": 0.01
}
params2 = {
    "input_dim": 10,
    "hidden_layers": [256, 128],
    "learning_rate": 0.01
}



stacking 2 DNN - smote

In [11]:
def binary(y):
    return (y > 0).astype(int)
res = 0
for j in range(len(file) - 1):
    for i in range(len(names[file[j]])):
        if i == len(names[file[j]]) - 1:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j+1]}/{file[j+1]}-{names[file[j+1]][0]}.csv"
        elif file[j] == 'xerces' and names[file[j]][i] == '1.4.4':
            break
        else:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i+1]}.csv"

        data_train = pd.read_csv(train_path)
        X_train = data_train.drop(['name', 'bug'], axis=1)
        y_train = binary(data_train['bug'])

        data_test = pd.read_csv(test_path)
        X_test = data_test.drop(['name', 'bug'], axis=1)
        y_test = binary(data_test['bug'])

        # Feature selection
        selector = SelectKBest(f_classif, k=10)
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)

        # Scaling
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train_selected)
        X_test_scaled = scaler.transform(X_test_selected)

        # Oversampling
        minority_class = min(Counter(y_train), key=Counter(y_train).get)
        minority_count = Counter(y_train)[minority_class]
        if minority_count <= 1:
            X_train_over, y_train_over = X_train_scaled, y_train
        else:
            k = min(5, minority_count - 1)
            oversample = SMOTE(k_neighbors=k)
            X_train_over, y_train_over = oversample.fit_resample(X_train_scaled, y_train)

        # Train DNN
        ensemble  = DNNStacking( params1, params2)
        ensemble.fit(X_train_over, y_train_over, epochs = 20, batch_size = 32)

        # Predict
        y_test_pred = (ensemble.predict(X_test_scaled) > 0.5).astype(int)
        y_train_pred = (ensemble.predict(X_train_over) > 0.5).astype(int)

        res += f1_score(y_test, y_test_pred)

        print(f"Training on {train_path.split('/')[-1]} & testing on {test_path.split('/')[-1]}: "
              f"f1_score = {f1_score(y_test, y_test_pred):.4f}    "
              f"f1_score_train = {f1_score(y_train_over, y_train_pred):.4f}")
print(res/37)


Epoch 1/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.5501 - loss: 0.6757 - val_accuracy: 0.0000e+00 - val_loss: 0.8062
Epoch 2/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5943 - loss: 0.5581 - val_accuracy: 0.0952 - val_loss: 1.3337
Epoch 3/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6917 - loss: 0.5757 - val_accuracy: 0.9524 - val_loss: 0.6634
Epoch 4/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7526 - loss: 0.4727 - val_accuracy: 0.8810 - val_loss: 0.6103
Epoch 5/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7594 - loss: 0.4677 - val_accuracy: 0.7143 - val_loss: 0.8616
Epoch 6/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8306 - loss: 0.4155 - val_accuracy: 0.8810 - val_loss: 0.4785
Epoch 7/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7866 - loss: 0.4663 - val_accuracy: 0.8333 - val_loss: 0.5913
Epoch 8/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8245 - loss: 0.4194 - val_accuracy: 0.6667 - val_loss: 0.7

stacking DNN - boot - smote

In [12]:
def binary(y):
    return (y > 0).astype(int)
res = 0
for j in range(len(file) - 1):
    for i in range(len(names[file[j]])):
        if i == len(names[file[j]]) - 1:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j+1]}/{file[j+1]}-{names[file[j+1]][0]}.csv"
        elif file[j] == 'xerces' and names[file[j]][i] == '1.4.4':
            break
        else:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i+1]}.csv"

        data_train = pd.read_csv(train_path)
        X_train = data_train.drop(['name', 'bug'], axis=1)
        y_train = binary(data_train['bug'])

        data_test = pd.read_csv(test_path)
        X_test = data_test.drop(['name', 'bug'], axis=1)
        y_test = binary(data_test['bug'])

        # Feature selection
        selector = SelectKBest(f_classif, k=10)
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)

        # Scaling
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train_selected)
        X_test_scaled = scaler.transform(X_test_selected)

        # Oversampling
        minority_class = min(Counter(y_train), key=Counter(y_train).get)
        minority_count = Counter(y_train)[minority_class]
        if minority_count <= 1:
            X_train_over, y_train_over = X_train_scaled, y_train
        else:
            k = min(5, minority_count - 1)
            oversample = SMOTE(k_neighbors=k)
            X_train_over, y_train_over = oversample.fit_resample(X_train_scaled, y_train)

        train_df = pd.concat([pd.DataFrame(X_train_over),
                              pd.Series(y_train_over, name="bug")], axis=1)
        major = train_df[train_df['bug'] == 0]
        minor = train_df[train_df['bug'] == 1]

        minority_unsampled = resample(minor,
                                      replace=True,
                                      n_samples=len(major),
                                      random_state=42)
        train_bootstrap = pd.concat([major, minority_unsampled])

        X_train_boot = train_bootstrap.drop('bug', axis=1).reset_index(drop=True)
        y_train_boot = train_bootstrap['bug'].reset_index(drop=True)


        # Train DNN
        ensemble  = DNNStacking( params1, params2)
        ensemble.fit(X_train_boot, y_train_boot, epochs = 20, batch_size = 32)

        # Predict
        y_test_pred = (ensemble.predict(X_test_scaled) > 0.5).astype(int)
        y_train_pred = (ensemble.predict(X_train_boot) > 0.5).astype(int)

        res += f1_score(y_test, y_test_pred)

        print(f"Training on {train_path.split('/')[-1]} & testing on {test_path.split('/')[-1]}: "
              f"f1_score = {f1_score(y_test, y_test_pred):.4f}    "
              f"f1_score_train = {f1_score(y_train_boot, y_train_pred):.4f}")
print(res/37)


Epoch 1/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.5944 - loss: 0.6405 - val_accuracy: 0.9524 - val_loss: 0.3540
Epoch 2/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7779 - loss: 0.4913 - val_accuracy: 0.7619 - val_loss: 0.5804
Epoch 3/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8449 - loss: 0.4150 - val_accuracy: 0.8333 - val_loss: 0.5100
Epoch 4/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8387 - loss: 0.4574 - val_accuracy: 0.8333 - val_loss: 0.4427
Epoch 5/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8414 - loss: 0.3894 - val_accuracy: 0.7381 - val_loss: 0.4523
Epoch 6/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8626 - loss: 0.3858 - val_accuracy: 0.8333 - val_loss: 0.4417
Epoch 7/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8676 - loss: 0.3669 - val_accuracy: 0.8571 - val_loss: 0.3719
Epoch 8/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8355 - loss: 0.3986 - val_accuracy: 0.8095 - val_loss: 0.4626


2 DNN - boot

In [11]:
def binary(y):
    return (y > 0).astype(int)
res = 0
for j in range(len(file) - 1):
    for i in range(len(names[file[j]])):
        if i == len(names[file[j]]) - 1:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j+1]}/{file[j+1]}-{names[file[j+1]][0]}.csv"
        elif file[j] == 'xerces' and names[file[j]][i] == '1.4.4':
            break
        else:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i+1]}.csv"

        data_train = pd.read_csv(train_path)
        X_train = data_train.drop(['name', 'bug'], axis=1)
        y_train = binary(data_train['bug'])

        data_test = pd.read_csv(test_path)
        X_test = data_test.drop(['name', 'bug'], axis=1)
        y_test = binary(data_test['bug'])

        # Feature selection
        selector = SelectKBest(f_classif, k=10)
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)

        # Scaling
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train_selected)
        X_test_scaled = scaler.transform(X_test_selected)

        
        train_df = pd.concat([pd.DataFrame(X_train_scaled),
                              pd.Series(y_train, name="bug")], axis=1)
        major = train_df[train_df['bug'] == 0]
        minor = train_df[train_df['bug'] == 1]

        minority_unsampled = resample(minor,
                                      replace=True,
                                      n_samples=len(major),
                                      random_state=42)
        train_bootstrap = pd.concat([major, minority_unsampled])

        X_train_boot = train_bootstrap.drop('bug', axis=1).reset_index(drop=True)
        y_train_boot = train_bootstrap['bug'].reset_index(drop=True)


        # Train DNN
        ensemble  = DNNStacking( params1, params2)
        ensemble.fit(X_train_boot, y_train_boot, epochs = 20, batch_size = 32)

        # Predict
        y_test_pred = (ensemble.predict(X_test_scaled) > 0.5).astype(int)
        y_train_pred = (ensemble.predict(X_train_boot) > 0.5).astype(int)

        res += f1_score(y_test, y_test_pred)

        print(f"Training on {train_path.split('/')[-1]} & testing on {test_path.split('/')[-1]}: "
              f"f1_score = {f1_score(y_test, y_test_pred):.4f}    "
              f"f1_score_train = {f1_score(y_train_boot, y_train_pred):.4f}")
print(res/37)


Epoch 1/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.5792 - loss: 0.6780 - val_accuracy: 0.8333 - val_loss: 0.5118
Epoch 2/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7500 - loss: 0.5337 - val_accuracy: 0.3810 - val_loss: 1.2470
Epoch 3/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7475 - loss: 0.5179 - val_accuracy: 0.5476 - val_loss: 0.7284
Epoch 4/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8123 - loss: 0.4098 - val_accuracy: 0.4762 - val_loss: 1.0774
Epoch 5/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7663 - loss: 0.4456 - val_accuracy: 0.5476 - val_loss: 0.8133
Epoch 6/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8542 - loss: 0.4061 - val_accuracy: 0.5476 - val_loss: 0.8776
Epoch 7/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8384 - loss: 0.4575 - val_accuracy: 0.5476 - val_loss: 0.7783
Epoch 8/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7957 - loss: 0.4288 - val_accuracy: 0.5476 - val_loss: 0.6684


2 DNN - boot - coral

In [ ]:
def binary(y):
    return (y > 0).astype(int)
res = 0
for j in range(len(file) - 1):
    for i in range(len(names[file[j]])):
        if i == len(names[file[j]]) - 1:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j+1]}/{file[j+1]}-{names[file[j+1]][0]}.csv"
        elif file[j] == 'xerces' and names[file[j]][i] == '1.4.4':
            break
        else:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i+1]}.csv"

        data_train = pd.read_csv(train_path)
        X_train = data_train.drop(['name', 'bug'], axis=1)
        y_train = binary(data_train['bug'])

        data_test = pd.read_csv(test_path)
        X_test = data_test.drop(['name', 'bug'], axis=1)
        y_test = binary(data_test['bug'])

        # Feature selection
        selector = SelectKBest(f_classif, k=10)
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)

        # Scaling
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train_selected)
        X_test_scaled = scaler.transform(X_test_selected)

        if i == len(names[file[j]]) - 1:
            X_train_scaled = Coral(X_train_scaled, X_test_scaled)



        
        train_df = pd.concat([pd.DataFrame(X_train_scaled),
                              pd.Series(y_train, name="bug")], axis=1)
        major = train_df[train_df['bug'] == 0]
        minor = train_df[train_df['bug'] == 1]

        minority_unsampled = resample(minor,
                                      replace=True,
                                      n_samples=len(major),
                                      random_state=42)
        train_bootstrap = pd.concat([major, minority_unsampled])

        X_train_boot = train_bootstrap.drop('bug', axis=1).reset_index(drop=True)
        y_train_boot = train_bootstrap['bug'].reset_index(drop=True)


        # Train DNN
        ensemble  = DNNStacking( params1, params2)
        ensemble.fit(X_train_boot, y_train_boot, epochs = 20, batch_size = 32)

        # Predict
        y_test_pred = (ensemble.predict(X_test_scaled) > 0.5).astype(int)
        y_train_pred = (ensemble.predict(X_train_boot) > 0.5).astype(int)

        res += f1_score(y_test, y_test_pred)

        print(f"Training on {train_path.split('/')[-1]} & testing on {test_path.split('/')[-1]}: "
              f"f1_score = {f1_score(y_test, y_test_pred):.4f}    "
              f"f1_score_train = {f1_score(y_train_boot, y_train_pred):.4f}")
print(res/37)


2 DNN - over - coral

In [ ]:
from sklearn.preprocessing import StandardScaler

def binary(y):
    return (y > 0).astype(int)
res = 0
for j in range(len(file) - 1):
    for i in range(len(names[file[j]])):
        if i == len(names[file[j]]) - 1:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j+1]}/{file[j+1]}-{names[file[j+1]][0]}.csv"
        elif file[j] == 'xerces' and names[file[j]][i] == '1.4.4':
            break
        else:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i+1]}.csv"

        data_train = pd.read_csv(train_path)
        X_train = data_train.drop(['name', 'bug'], axis=1)
        y_train = binary(data_train['bug'])

        data_test = pd.read_csv(test_path)
        X_test = data_test.drop(['name', 'bug'], axis=1)
        y_test = binary(data_test['bug'])

        # Feature selection
        selector = SelectKBest(f_classif, k=10)
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)

        # Scaling
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train_selected)
        X_test_scaled = scaler.transform(X_test_selected)

        if i == len(names[file[j]]) - 1:
            X_train_scaled = Coral(X_train_scaled, X_test_scaled)


        # Oversampling
        minority_class = min(Counter(y_train), key=Counter(y_train).get)
        minority_count = Counter(y_train)[minority_class]
        if minority_count <= 1:
            X_train_over, y_train_over = X_train_scaled, y_train
        else:
            k = min(5, minority_count - 1)
            oversample = SMOTE(k_neighbors=k)
            X_train_over, y_train_over = oversample.fit_resample(X_train_scaled, y_train)

        


        # Train DNN
        ensemble  = DNNStacking( params1, params2)
        ensemble.fit(X_train_boot, y_train_boot, epochs = 20, batch_size = 32)

        # Predict
        y_test_pred = (ensemble.predict(X_test_scaled) > 0.5).astype(int)
        y_train_pred = (ensemble.predict(X_train_over) > 0.5).astype(int)

        res += f1_score(y_test, y_test_pred)

        print(f"Training on {train_path.split('/')[-1]} & testing on {test_path.split('/')[-1]}: "
              f"f1_score = {f1_score(y_test, y_test_pred):.4f}    "
              f"f1_score_train = {f1_score(y_train_over, y_train_pred):.4f}")
print(res/37)


Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 700ms/step - accuracy: 0.2353 - loss: 0.7001 - val_accuracy: 0.0000e+00 - val_loss: 0.8915
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.6471 - loss: 0.6441 - val_accuracy: 0.0000e+00 - val_loss: 0.9357
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.6471 - loss: 0.6175 - val_accuracy: 0.4000 - val_loss: 0.7712
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.7059 - loss: 0.5882 - val_accuracy: 0.8000 - val_loss: 0.5953
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.7647 - loss: 0.5587 - val_accuracy: 0.8000 - val_loss: 0.4670
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.7059 - loss: 0.5319 - val_accuracy: 1.0000 - val_loss: 0.3803
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.7059 - loss: 0.5001 - val_accuracy: 1.0000 - val_loss: 0.3600
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.7059 - loss: 0.4606 - val_accuracy: 0.8000 - val_loss

CNN stacking  - smote - coral

In [ ]:
from tensorflow.keras import layers, models
from sklearn.linear_model import LogisticRegression
import numpy as np

# CNN cơ bản
def CNN(input_shape):
    model = models.Sequential()
    model.add(layers.Conv1D(32, kernel_size=3, activation='relu', padding="same", input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv1D(64, kernel_size=3, activation='relu', padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.GlobalMaxPooling1D())

    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# CNN sâu 5 lớp
def CNN5(input_shape):
    model = models.Sequential()
    model.add(layers.Conv1D(32, kernel_size=3, activation='relu', padding="same", input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv1D(64, kernel_size=3, activation='relu', padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv1D(128, kernel_size=3, activation='relu', padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv1D(256, kernel_size=3, activation='relu', padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv1D(512, kernel_size=3, activation='relu', padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))

    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

class CNNStacking:
    def __init__(self, input_shape, epochs=10, batch_size=32):
        self.model1 = CNN(input_shape)
        self.model2 = CNN5(input_shape)
        self.meta = LogisticRegression()
        self.epochs = epochs
        self.batch_size = batch_size

    def fit(self, X_train, y_train):
        self.model1.fit(X_train, y_train, epochs=self.epochs, batch_size=self.batch_size, verbose=0)
        self.model2.fit(X_train, y_train, epochs=self.epochs, batch_size=self.batch_size, verbose=0)
        pred1 = self.model1.predict(X_train).reshape(-1, 1)
        pred2 = self.model2.predict(X_train).reshape(-1, 1)
        X_meta = np.hstack([pred1, pred2])

        self.meta.fit(X_meta, y_train.ravel()) 

    def predict(self, X):
        pred1 = self.model1.predict(X).reshape(-1, 1)
        pred2 = self.model2.predict(X).reshape(-1, 1)
        X_meta = np.hstack([pred1, pred2])
        return self.meta.predict_proba(X_meta)[:, 1]


In [20]:


def binary(y):
    return (y > 0).astype(int)
res = 0 
for j in range(len(file) - 1):
    for i in range(len(names[file[j]])):

        if i == len(names[file[j]]) - 1:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j+1]}/{file[j+1]}-{names[file[j+1]][0]}.csv"
        elif file[j] == 'xerces' and names[file[j]][i] == '1.4.4':
            break
        else:
            train_path = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i]}.csv"
            test_path  = f"C:/Users/ACER/Downloads/{file[j]}/{file[j]}-{names[file[j]][i+1]}.csv"

        data_train = pd.read_csv(train_path)
        X_train = data_train.drop(['name', 'bug'], axis=1)
       
        data_test = pd.read_csv(test_path)
        X_test = data_test.drop(['name', 'bug'], axis=1)


        y_train = binary(data_train['bug'])
        y_test = binary(data_test['bug'])


        selector = SelectKBest(f_classif, k=10)
        X_train_selected = selector.fit_transform(X_train, y_train)
        X_test_selected = selector.transform(X_test)

        
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train_selected)
        X_test = scaler.transform(X_test_selected)

        if i == len(names[file[j]]) - 1:
            
            X_train = Coral(X_train, X_test)


        

        minority_class = min(Counter(y_train), key=Counter(y_train).get)
        minority_count = Counter(y_train)[minority_class]

        if minority_count <= 1:
            X_train_smote, y_train_smote = X_train, y_train
        else:
            k = min(5, minority_count - 1)
            smote = SMOTE(k_neighbors=k, random_state=42)
            X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

        X_train_cnn = np.expand_dims(X_train_smote, axis=2)
        X_test_cnn  = np.expand_dims(X_test, axis=2)

    
        ensemble  = CNNStacking( input_shape  = (X_train_cnn.shape[1],1),
                                epochs = 12, batch_size= 32)
        
        ensemble.fit(X_train_cnn, y_train_smote)
    
        y_train_pred = (ensemble.predict(X_train_cnn) > 0.5).astype(int)
        y_test_pred  = (ensemble.predict(X_test_cnn) > 0.5).astype(int)
        res += f1_score(y_test, y_test_pred)
        print(f"Training on {train_path.split('/')[-1]} & testing on {test_path.split('/')[-1]}: "
              f"f1_score = {f1_score(y_test, y_test_pred):.4f}    "
              f"f1_score_train = {f1_score(y_train_smote, y_train_pred):.4f}")

print(res/37)

C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


C:\Users\ACER\AppData\Local\Temp\ipykernel_338512\3555117758.py:71: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  self.meta.fit(X_meta, y_train.ravel())  # ravel để y_train là 1D


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Training on ant-1.3.csv & testing on ant-1.4.csv: f1_score = 0.2936    f1_score_train = 0.7130


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Training on xalan-2.6.csv & testing on xalan-2.7.csv: f1_score = 0.5927    f1_score_train = 0.7060


C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
10/57 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 

C:\Users\ACER\AppData\Local\Temp\ipykernel_338512\3555117758.py:71: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  self.meta.fit(X_meta, y_train.ravel())  # ravel để y_train là 1D


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
Training on xalan-2.7.csv & testing on xerces-1.1.csv: f1_score = 0.4183    f1_score_train = 0.9498
0.42859217813592987
